In [1]:
%matplotlib inline

This US Census dataset contains detailed but anonymised information for approximately 300,000 people.

The archive contains 3 files:
o   A large learning .csv file
o   Another test .csv file
o   A metadata file describing the columns of the two above mentioned files (identical for both)

The goal of this exercise is to “modelize” / “predict” the information contained in the last column (42nd), i.e., which people save more or less than $50,000 / year, from the information contained in the other columns.
The exercise here consists of modelizing a binary variable.

# Load Libraries

In [2]:
from pprint import pprint
import matplotlib.pyplot as Plot
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from IPython.display import display, HTML
pd.set_option('display.max_columns', 50)

print('Loaded...')

Loaded...


# Read Data

In [3]:
learndf = pd.read_csv("census_income_learn.csv", header = None, skipinitialspace = True, na_values= "Not in universe")
testdf = pd.read_csv("census_income_test.csv", header = None, skipinitialspace = True, na_values= "Not in universe")

display(learndf)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,73,NaN,0,0,High school graduate,0,NaN,Widowed,Not in universe or children,NaN,White,All other,Female,NaN,NaN,Not in labor force,0,0,0,Nonfiler,NaN,NaN,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,NaN,United-States,United-States,United-States,Native- Born in the United States,0,NaN,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,NaN,Divorced,Construction,Precision production craft & repair,White,All other,Male,NaN,NaN,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,NaN,United-States,United-States,United-States,Native- Born in the United States,0,NaN,2,52,94,- 50000.
2,18,NaN,0,0,10th grade,0,High school,Never married,Not in universe or children,NaN,Asian or Pacific Islander,All other,Female,NaN,NaN,Not in labor force,0,0,0,Nonfiler,NaN,NaN,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,NaN,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,NaN,2,0,95,- 50000.
3,9,NaN,0,0,Children,0,NaN,Never married,Not in universe or children,NaN,White,All other,Female,NaN,NaN,Children or Armed Forces,0,0,0,Nonfiler,NaN,NaN,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,NaN,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,NaN,0,0,94,- 50000.
4,10,NaN,0,0,Children,0,NaN,Never married,Not in universe or children,NaN,White,All other,Female,NaN,NaN,Children or Armed Forces,0,0,0,Nonfiler,NaN,NaN,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,NaN,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,NaN,0,0,94,- 50000.
5,48,Private,40,10,Some college but no degree,1200,NaN,Married-civilian spouse present,Entertainment,Professional specialty,Amer Indian Aleut or Eskimo,All other,Female,No,NaN,Full-time schedules,0,0,0,Joint both under 65,NaN,NaN,Spouse of householder,Spouse of householder,162.61,?,?,?,Not in universe under 1 year old,?,1,NaN,Philippines,United-States,United-States,Native- Born in the United States,2,NaN,2,52,95,- 50000.
6,42,Private,34,3,Bachelors degree(BA AB BS),0,NaN,Married-civilian spouse present,Finance insurance and real estate,Executive admin and managerial,White,All other,Male,NaN,NaN,Children or Armed Forces,5178,0,0,Joint both under 65,NaN,NaN,Householder,Householder,1535.86,Nonmover,Nonmover,Nonmover,Yes,NaN,6,NaN,United-States,United-States,United-States,Native- Born in the United States,0,NaN,2,52,94,- 50000.
7,28,Private,4,40,High school graduate,0,NaN,Never married,Construction,Handlers equip cleaners etc,White,All other,Female,NaN,Job loser - on layoff,Unemployed full-time,0,0,0,Single,NaN,NaN,Secondary individual,Nonrelative of householder,898.83,?,?,?,Not in universe under 1 year old,?,4,NaN,United-States,United-States,United-States,Native- Born in the United States,0,NaN,2,30,95,- 50000.
8,47,Local government,43,26,Some college but no degree,876,NaN,Married-civilian spouse present,Education,Adm support including clerical,White,All other,Female,No,NaN,Full-time schedules,0,0,0,Joint both under 65,NaN,NaN,Spouse of householder,Spouse of householder,1661.53,?,?,?,Not in universe under 1 year old,?,5,NaN,United-States,United-States,United-States,Native- Born in the United States,0,NaN,2,52,95,- 50000.
9,34,Private,4,37,Some college but no degree,0,NaN,Married-civilian spouse present,Construction,Machine operators assmblrs & inspctrs,White,All other,Male,NaN,NaN,Children or Armed Forces,0,0,0,Joint both under 65,NaN,NaN,Householder,Householder,1146.79,Nonmover,Nonmover,Nonmover,Yes,NaN,6,NaN,United-States,United-Sta

# Analysis
Based on the learning file, make a quick statistic based and univariate audit of the different columns’ content and produce the results in visual / graphic format

In [4]:
learndf.describe()

,0,2,3,5,16,17,18,24,30,36,38,39,40
count,199523.000000,199523.000000,199523.000000,199523.000000,199523.00000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000
mean,34.494199,15.352320,11.306556,55.426908,434.71899,37.313788,197.529533,1740.380269,1.956180,0.175438,1.514833,23.174897,94.499672
std,22.310895,18.067129,14.454204,274.896454,4697.53128,271.896428,1984.163658,993.768156,2.365126,0.553694,0.851473,24.411488,0.500001
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,37.870000,0.000000,0.000000,0.000000,0.000000,94.000000
25%,15.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1061.615000,0.000000,0.000000,2.000000,0.000000,94.000000
50%,33.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1618.310000,1.000000,0.000000,2.000000,8.000000,94.000000
75%,50.000000,33.000000,26.000000,0.000000,0.00000,0.000000,0.000000,2188.610000,4.000000,0.000000,2.000000,52.000000,95.000000
max,90.000000,51.000000,46.000000,9999.000000,99999.00000,4608.000000,99999.000000,18656.300000,6.000000,2.000000,2.000000,52.000000,95.000000


In [5]:
for col in learndf.columns:
    print('{} : {} {}% NaN'.format(col, learndf[col].dtype, (learndf[col].isnull().sum() / len(learndf.index)) * 100))
    print(learndf[col].unique())

0 : int64 0.0% NaN
[73 58 18  9 10 48 42 28 47 34  8 32 51 46 26 13 39 16 35 12 27 56 55  2  1
 37  4 63 25 81 11 30  7 66 84 52  5 36 72 61 41 90 49  6  0 33 57 50 24 17
 53 40 54 22 29 85 38 76 21 31 74 19 15  3 43 68 71 45 62 23 69 75 44 59 60
 64 65 70 67 78 20 14 83 86 89 77 79 82 80 87 88]
1 : object 50.24232795216591% NaN
[nan 'Self-employed-not incorporated' 'Private' 'Local government'
 'Federal government' 'Self-employed-incorporated' 'State government'
 'Never worked' 'Without pay']
2 : int64 0.0% NaN
[ 0  4 40 34 43 37 24 39 12 35 45  3 19 29 32 48 33 23 44 36 31 30 41  5 11
  9 42  6 18 50  2  1 26 47 16 14 22 17  7  8 25 46 27 15 13 49 38 21 28 20
 51 10]
3 : int64 0.0% NaN
[ 0 34 10  3 40 26 37 31 12 36 41 22  2 35 25 23 42  8 19 29 27 16 33 13 18
  9 17 39 32 11 30 38 20  7 21 44 24 43 28  4  1  6 45 14  5 15 46]
4 : object 0.0% NaN
['High school graduate' 'Some college but no degree' '10th grade'
 'Children' 'Bachelors degree(BA AB BS)'
 'Masters degree(MA MS MEng MEd 

In [6]:
learndf.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
count,199523.000000,99278,199523.000000,199523.000000,199523,199523.000000,12580,199523,199523,98839,199523,198649,199523,19064,6070,199523,199523.00000,199523.000000,199523.000000,199523,15773,15773,199523,199523,199523.000000,198007,198007,198007,199523,115469,199523.000000,55291,199523,199523,199523,199523,199523.000000,1984,199523.000000,199523.000000,199523.000000,199523
unique,NaN,8,NaN,NaN,17,NaN,2,7,24,14,5,9,2,2,5,8,NaN,NaN,NaN,6,5,50,38,8,NaN,9,8,9,3,3,NaN,4,43,43,43,5,NaN,2,NaN,NaN,NaN,2
top,NaN,Private,NaN,NaN,High school graduate,NaN,High school,Never married,Not in universe or children,Adm support including clerical,White,All other,Female,No,Other job loser,Children or Armed Forces,NaN,NaN,NaN,Nonfiler,South,California,Householder,Householder,NaN,?,?,?,Not in universe under 1 year old,?,NaN,Both parents present,United-States,United-States,United-States,Native- Born in the United States,NaN,No,NaN,NaN,NaN,- 50000.
freq,NaN,72028,NaN,NaN,48407,NaN,6892,86485,100684,14837,167365,171907,103984,16034,2038,123769,NaN,NaN,NaN,75094,4889,1714,53248,75475,NaN,99696,99696,99696,101212,99696,NaN,38983,159163,160479,176989,176992,NaN,1593,NaN,NaN,NaN,187141
mean,34.494199,NaN,15.352320,11.306556,NaN,55.426908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,434.71899,37.313788,197.529533,NaN,NaN,NaN,NaN,NaN,1740.380269,NaN,NaN,NaN,NaN,NaN,1.956180,NaN,NaN,NaN,NaN,NaN,0.175438,NaN,1.514833,23.174897,94.499672,NaN
std,22.310895,NaN,18.067129,14.454204,NaN,274.896454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4697.53128,271.896428,1984.163658,NaN,NaN,NaN,NaN,NaN,993.768156,NaN,NaN,NaN,NaN,NaN,2.365126,NaN,NaN,NaN,NaN,NaN,0.553694,NaN,0.851473,24.411488,0.500001,NaN
min,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,37.870000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,94.000000,NaN
25%,15.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1061.615000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,0.000000,94.000000,NaN
50%,33.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1618.310000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,8.000000,94.000000,NaN
75%,50.000000,NaN,33.000000,26.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,2188.610000,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,52.000000,95.000000,NaN


# Procedural Comments

Having taken a look at the data, it is obvious to me that the columns might not necessarily be the columns indicated in the metadata file.  Other than assuming that column 0 = age and column 41 is our prediction column, I made a decision to assume that the remaining data was mislabeled. 

In [7]:
def isOver(row):
    return 0 if row[len(learndf.columns)-1] == '- 50000.' else 1

def validate(df):
    assert isinstance(df, pd.DataFrame)
    for col in df:
        if df[col].isnull().sum() > 0:
            print("Error NaN detected for {}!".format(col))
            return False
    print("No NaNs.")
    return True

def print_scores(model, X_test, y_true, y_pred):
    if y_pred.dtype == np.float16 or y_pred.dtype == np.float32 or y_pred.dtype == np.float64:
        y_pred = GetPrediction(y_pred)
    acc_score_norm = metrics.accuracy_score(y_true, y_pred)
    acc_score_non_norm = metrics.accuracy_score(y_true, y_pred, normalize=False)
    print('Acc norm: {} Acc non-norm: {}'.format(acc_score_norm, acc_score_non_norm))
    ce_score_norm = metrics.log_loss(y_true, y_pred)
    ce_score_non_norm = metrics.log_loss(y_true, y_pred, normalize=False)
    print('CE norm: {} CE non-norm: {}'.format(ce_score_norm, ce_score_non_norm))
    matthews = metrics.matthews_corrcoef(y_true, y_pred)
    print('Matthews Cor. Coef: {}'.format(matthews))
    scores = roc_auc(model, X_test, y_true, y_pred)
    print('roc_auc: {} <- {}'.format(np.average(scores), scores))

def get_roc_auc(model, X_test, y_true, y_pred):
    scores = cross_val_score(model, X_test, y=y_true, scoring='roc_auc', n_jobs=-1)
    return scores

print('Loaded...')

Loaded...


# Build a model

In [11]:
y_train = pd.DataFrame()
y_train['IsOver'] = learndf.apply(isOver, axis=1)
y_test = pd.DataFrame()
y_test['IsOver'] = testdf.apply(isOver, axis=1)
del learndf[len(learndf.columns)-1]
del testdf[len(learndf.columns)-1]

y_train.describe()

,IsOver
count,199523.000000
mean,0.062058
std,0.241261
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [12]:
model = LogisticRegression()
X_train = pd.DataFrame()
X_test = pd.DataFrame()
roc_auc = 0
roc_auc_list = [roc_auc]

for col in learndf:
    if learndf[col].dtype.name == 'int64' or learndf[col].dtype.name == 'float64' :
        col_train = pd.DataFrame({col : learndf[col]})
        col_test = pd.DataFrame({col : testdf[col]})
    elif learndf[col].dtype.name == 'object':
        col_train = pd.get_dummies(learndf[col])
        col_test = pd.get_dummies(testdf[col])
    else:
        print('bad type')
        () + 1
    assert isinstance(col_train, pd.DataFrame)
    assert isinstance(col_test, pd.DataFrame)
    if len(col_train.columns) != len(col_test.columns):
        for newcol in col_train:
            if newcol not in col_test.columns:
                col_test[newcol] = 0
    new_X_train = pd.concat([X_train, col_train], axis=1)
    new_X_test = pd.concat([X_test, col_test], axis=1)
    model.fit(new_X_train, y_train.IsOver.ravel())
    y_pred = model.predict(new_X_test)
    scores = get_roc_auc(model, new_X_test, y_test.IsOver.values, y_pred)
    new_roc_auc = np.average(scores)
    print('{} roc_auc: {} <- {}'.format(col, np.average(scores), scores))
    if new_roc_auc > roc_auc:
        X_train = new_X_train
        X_test = new_X_test
        roc_auc = new_roc_auc
        roc_auc_list.append(roc_auc)

y_pred = model.predict(X_test)
print_scores(model, X_test, y_test.IsOver.values, y_pred)

0 roc_auc: 0.6962662371584746 <- [ 0.69516979  0.69543016  0.69819876]
1 roc_auc: 0.8271045467111823 <- [ 0.83043102  0.82235843  0.82852419]
2 roc_auc: 0.827007000000083 <- [ 0.83042162  0.82208099  0.82851839]
3 roc_auc: 0.8651394019307596 <- [ 0.86623623  0.86326569  0.86591629]
4 roc_auc: 0.8942116490612008 <- [ 0.89555303  0.89333558  0.89374634]
5 roc_auc: 0.8941994932960916 <- [ 0.89552634  0.89335861  0.89371354]
6 roc_auc: 0.8957623499878563 <- [ 0.89709971  0.89502242  0.89516492]
7 roc_auc: 0.8992396606498039 <- [ 0.90057514  0.89907931  0.89806453]
8 roc_auc: 0.909871016111885 <- [ 0.9108068   0.91008991  0.90871634]
9 roc_auc: 0.9160843715719759 <- [ 0.91785551  0.91643796  0.91395965]
10 roc_auc: 0.9160677458228945 <- [ 0.9180424   0.91638785  0.91377299]
11 roc_auc: 0.9161106213908962 <- [ 0.91771916  0.91661544  0.91399726]
12 roc_auc: 0.9234457484808584 <- [ 0.926054    0.92405433  0.92022891]
13 roc_auc: 0.9236206246190278 <- [ 0.92624198  0.92422133  0.92039857]
14 r

KeyError: 40

# Model Analysis

In [44]:
X_train.describe(include='all')

Acc norm: 0.9391351416370963 Acc non-norm: 93690
CE norm: 2.1022029831620705 CE non-norm: 209719.9740062145
Matthews Cor. Coef: 0.23851670022483315
roc_auc: 0.8875778974389466 <- [ 0.88924118  0.89113586  0.88235665]


In [28]:
#ToDo: Graph roc_auc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
count,199523.000000,99278,199523.000000,199523.000000,199523,199523.000000,12580,199523,199523,98839,199523,198649,199523,19064,6070,199523,199523.00000,199523.000000,199523.000000,199523,15773,15773,199523,199523,199523.000000,198007,198007,198007,199523,115469,199523.000000,55291,199523,199523,199523,199523,199523.000000,1984,199523.000000,199523.000000,199523.000000,199523
unique,NaN,8,NaN,NaN,17,NaN,2,7,24,14,5,9,2,2,5,8,NaN,NaN,NaN,6,5,50,38,8,NaN,9,8,9,3,3,NaN,4,43,43,43,5,NaN,2,NaN,NaN,NaN,2
top,NaN,Private,NaN,NaN,High school graduate,NaN,High school,Never married,Not in universe or children,Adm support including clerical,White,All other,Female,No,Other job loser,Children or Armed Forces,NaN,NaN,NaN,Nonfiler,South,California,Householder,Householder,NaN,?,?,?,Not in universe under 1 year old,?,NaN,Both parents present,United-States,United-States,United-States,Native- Born in the United States,NaN,No,NaN,NaN,NaN,- 50000.
freq,NaN,72028,NaN,NaN,48407,NaN,6892,86485,100684,14837,167365,171907,103984,16034,2038,123769,NaN,NaN,NaN,75094,4889,1714,53248,75475,NaN,99696,99696,99696,101212,99696,NaN,38983,159163,160479,176989,176992,NaN,1593,NaN,NaN,NaN,187141
mean,34.494199,NaN,15.352320,11.306556,NaN,55.426908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,434.71899,37.313788,197.529533,NaN,NaN,NaN,NaN,NaN,1740.380269,NaN,NaN,NaN,NaN,NaN,1.956180,NaN,NaN,NaN,NaN,NaN,0.175438,NaN,1.514833,23.174897,94.499672,NaN
std,22.310895,NaN,18.067129,14.454204,NaN,274.896454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4697.53128,271.896428,1984.163658,NaN,NaN,NaN,NaN,NaN,993.768156,NaN,NaN,NaN,NaN,NaN,2.365126,NaN,NaN,NaN,NaN,NaN,0.553694,NaN,0.851473,24.411488,0.500001,NaN
min,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,37.870000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,94.000000,NaN
25%,15.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1061.615000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,0.000000,94.000000,NaN
50%,33.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,1618.310000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,8.000000,94.000000,NaN
75%,50.000000,NaN,33.000000,26.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,2188.610000,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2.000000,52.000000,95.000000,NaN


In [27]:
learndf.isnull().sum()

0          0
1     100245
2          0
3          0
4          0
5          0
6     186943
7          0
8          0
9     100684
10         0
11       874
12         0
13    180459
14    193453
15         0
16         0
17         0
18         0
19         0
20    183750
21    183750
22         0
23         0
24         0
25      1516
26      1516
27      1516
28         0
29     84054
30         0
31    144232
32         0
33         0
34         0
35         0
36         0
37    197539
38         0
39         0
40         0
41         0
dtype: int64